In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Redis Sets Explained - [Link to Video](https://youtu.be/6Pjz819sT7M)

### What a Redis Set Is

A Redis Set is an unordered collection of unique strings with no duplicates; adding the same member repeatedly results in a single stored member. Sets support O(1) membership checks and provide standard set operations like intersection, union, and difference ([Redis Sets — docs](https://redis.io/docs/latest/develop/data-types/sets/), [SISMEMBER — docs](https://redis.io/docs/latest/commands/sismember/)).

```python
# create two demo sets (idempotent additions; duplicates are ignored)
r.execute_command("SADD", "demo:set:a", "a", "b", "c", "c", "a")
r.execute_command("SADD", "demo:set:b", "b", "c", "d")

# verify uniqueness and unordered nature by checking cardinality and membership
print("SCARD demo:set:a ->", r.execute_command("SCARD", "demo:set:a"))        # 3
print("SISMEMBER demo:set:a 'c' ->", r.execute_command("SISMEMBER", "demo:set:a", "c"))  # 1
````

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/sets/](https://redis.io/docs/latest/develop/data-types/sets/)
>
> **Concept**: `Redis Set` → Unordered collection of unique string members with constant-time membership checks.
>
> **Context**: Used for de-duplication, membership queries, and set algebra across keys.
>
> **Example**: Tracking unique IPs or online users by ID.
>
> **Implication**:
>
> Constant-time checks and duplicate suppression enable efficient presence and de-duplication workflows.

---

### Use Case: “Who’s Online” Widget (Game Presence)

The game client pings the server every minute. On each ping, the server adds the player ID to a shared `players:online` Set. To display “who’s online,” the UI uses Set size (cardinality), membership checks for specific users, and intersection between a user’s `friends` Set and `players:online`.

```python
# add a player to the online set on heartbeat (idempotent if already present)
r.execute_command("SADD", "players:online", "42")

# verify this player is recorded as online
print("SISMEMBER players:online 42 ->", r.execute_command("SISMEMBER", "players:online", "42"))  # 1
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sismember/](https://redis.io/docs/latest/commands/sismember/)
>
> **Concept**: `O(1) Membership Test` → Checking whether an element is in a Set is constant time.
>
> **Context**: Powering the “Online” indicator for a player profile.
>
> **Example**: `SISMEMBER players:online 42` returns `1` if player 42 is online.
>
> **Implication**:
>
> Efficient per-user presence checks at scale.

---

### Count How Many Players Are Online

Use `SCARD` to get the number of elements in `players:online`.

```python
# populate online set with sample IDs
r.execute_command("SADD", "players:online", "1", "42", "101", "7", "8", "9", "256", "512")

# get total count of online players
print("SCARD players:online ->", r.execute_command("SCARD", "players:online"))  # e.g., 8
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/scard/](https://redis.io/docs/latest/commands/scard/)
>
> **Command**: `SCARD` → Returns the cardinality (size) of the Set.
>
> **Pattern**: `SCARD key`
>
> **Example**: `SCARD players:online`
>
> **Result**:
>
> Integer representing the number of members (O(1)).

---

### Check If a Specific User Is Online

Use `SISMEMBER` to test membership in `players:online`.

```python
# check a user that is online
print("SISMEMBER players:online 42 ->", r.execute_command("SISMEMBER", "players:online", "42"))  # 1

# check a user that is not online
print("SISMEMBER players:online 32 ->", r.execute_command("SISMEMBER", "players:online", "32"))  # 0
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sismember/](https://redis.io/docs/latest/commands/sismember/)
>
> **Command**: `SISMEMBER` → Returns whether a value exists in a Set.
>
> **Pattern**: `SISMEMBER key member`
>
> **Example**: `SISMEMBER players:online 42`
>
> **Result**:
>
> `1` if present, `0` otherwise (O(1)).

---

### Store a Player’s Friends

Each player’s friend list is stored as a Set, e.g., `player:31:friends`.

```python
# store friend IDs (includes 42 and 101 from the online set)
r.execute_command("SADD", "player:31:friends", "42", "101", "7", "8", "9", "33")

# verify friend list membership and size
print("SISMEMBER player:31:friends 101 ->", r.execute_command("SISMEMBER", "player:31:friends", "101"))  # 1
print("SCARD player:31:friends ->", r.execute_command("SCARD", "player:31:friends"))  # e.g., 6
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sadd/](https://redis.io/docs/latest/commands/sadd/)
>
> **Command**: `SADD` → Adds one or more members to a Set (duplicates ignored; creates Set if absent).
>
> **Pattern**: `SADD key member [member ...]`
>
> **Example**: `SADD player:31:friends 42 101 7 8 9 33`
>
> **Result**:
>
> Integer count of new members added (O(1) per element).

---

### Compute “Friends Who Are Online” (Set Intersection)

Use `SINTER` to intersect `player:31:friends` and `players:online`, returning only friends currently online.

```python
# intersect friends with the online set to find online friends
online_friends = r.execute_command("SINTER", "player:31:friends", "players:online")
print("SINTER player:31:friends players:online ->", online_friends)  # e.g., ['42', '101', '7', '8', '9']
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sinter/](https://redis.io/docs/latest/commands/sinter/)
>
> **Command**: `SINTER` → Returns members common to all provided Sets.
>
> **Pattern**: `SINTER key [key ...]`
>
> **Example**: `SINTER player:31:friends players:online`
>
> **Result**:
>
> Array of members in the intersection (worst-case complexity O(N·M), where N is the size of the smallest Set and M the number of Sets).

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/sets/](https://redis.io/docs/latest/develop/data-types/sets/)
>
> **Concept**: `Set Intersection` → The subset of members present in all compared Sets.
>
> **Context**: Resolving “which friends are online” by intersecting the friend list with the global online Set.
>
> **Example**: `SINTER friends online` returns friend IDs currently online.
>
> **Implication**:
>
> Enables efficient, server-side filtering for real-time UI elements.

---

### Caveat: Logouts and Stale Membership

If a player logs off and the ID is not actively removed, it remains in `players:online`, which can make the set stale. Handling techniques for removal/expiration are addressed separately.

```python
# simulate a user going offline without removal; membership remains true
r.execute_command("SADD", "players:online", "999")  # user 999 was online
print("SISMEMBER players:online 999 ->", r.execute_command("SISMEMBER", "players:online", "999"))  # 1 (stale if not removed)
```

---

### Command Reference (Technical)

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sadd/](https://redis.io/docs/latest/commands/sadd/)
>
> **Command**: `SADD` → Add set members (ignore duplicates).
>
> **Pattern**: `SADD key member [member ...]`
>
> **Example**: `SADD players:online 42`
>
> **Result**:
>
> New members count; O(1) per element.

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/scard/](https://redis.io/docs/latest/commands/scard/)
>
> **Command**: `SCARD` → Set cardinality (size).
>
> **Pattern**: `SCARD key`
>
> **Example**: `SCARD players:online`
>
> **Result**:
>
> Integer size; O(1).

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sismember/](https://redis.io/docs/latest/commands/sismember/)
>
> **Command**: `SISMEMBER` → Membership test.
>
> **Pattern**: `SISMEMBER key member`
>
> **Example**: `SISMEMBER players:online 32`
>
> **Result**:
>
> `1` if present, `0` otherwise; O(1).

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/sinter/](https://redis.io/docs/latest/commands/sinter/)
>
> **Command**: `SINTER` → Set intersection across keys.
>
> **Pattern**: `SINTER key [key ...]`
>
> **Example**: `SINTER player:31:friends players:online`
>
> **Result**:
>
> Array of common members; worst-case O(N·M).


### Original Transcript

Video title: Redis Sets Explained Video URL: https://youtu.be/6Pjz819sT7M Video language: English (United States) -------------------------------- How well do you know sets, and what can you use them for in a database like Redis? Sets are incredibly powerful. In this video, I'm going to tell you what Redis sets are, and then I'll show you how to use Redis sets to build a Who's Online widget for a multiplayer game. Let's dive in. [MUSIC PLAYING] So first, what is a set exactly? In Redis, a set is an unordered collection of strings that contains no duplicates. So if you add the same value to an empty set 20 times or even a million times, the set will contain just one member. This makes sets a natural fit for tasks like de-duplication. Another important set property? Efficient membership checks. Questions like, "Did I see this IP address in the last hour? Is this user online? Has this URL been blacklisted?" can all be answered in 0 of 1 time. Last but not least, Redis sets support standard mathematical set operations, like intersection, difference, and union. We're going to see a very interesting use of set intersection later on. So keep watching. Now, let's tie these concepts together with an example. In the online role-playing game Mages & Minotaurs, players go on quests, kill monsters, and collect treasure. Now, it's always more fun to play Mages & Minotaurs with your friends. So let's see how we can use sets to let our users see which of their friends are online. Now, the Mages & Minotaurs game client pings the game server every minute. When the server receives the request, it adds the player's ID to a set. Here's how we add a player's ID to the players:online set. We call the command SADD followed by the set name. So players:online followed by the player's ID, which in this case is 42. And the command returns 1. That's the number of players added to this set. Now imagine our players online set is full of IDs. How do we go about showing how many players are online? For that, we can use SCARD to get the cardinality or number of members in the set. Here, we're calling SCARD with the set name players:online. And you can see the command returns... a BILLION?! Guys, we have a billion players online. Yes, yes! Another thing our players online set gives us is the ability to efficiently find out whether any given user is online. If we're viewing a friend's profile, the server will call SISMEMBER to determine whether to light up the "I'm online" button. Let's see if player 42 is online. We'll call SISMEMBER followed by the name of the set, which again is players:online. We also give SISMEMBER the player's ID. You can see that player 42 is online because Redis returns 1. What about player 32? Nope. Redis returns 0, which means player 32 is not in the set and consequently not online. Of course, in a game like Mages & Minotaurs, we want to play with our friends. In Redis, we'd store each of our friends' IDs in a set. Here's the content of my friends set. You see my good friend Artexius is there. He's player 42. And so is my BFF, Sheila, player 101. So take a moment to think about this. If all my friends' IDs are now stored in a set, and we also have a set containing everyone who's online, what's the best way to find out which of my friends are online? The answer is set intersection. Now, if you aren't familiar with this term, the intersection of two sets is the subset of members in both sets. And the Redis command for this is SINTER. So to get my friends who are online, the server passes SINTER, the set name player:31:friends and the set name players:online. You can see that five of my friends are online. We've kept this example simple to focus on the most important set commands, but you should now have a general idea of how you would build a Who's Online widget for Mages & Minotaurs. But consider what happens if a player logs off the game? With our current approach, that player's ID will still be in the set of online players. I'll show you a few techniques for solving this problem in part 2 of this video. So to review, we learned that a set is an unordered collection of strings that contains no duplicates. And we reviewed the most important set commands. We learned about SADD, which adds a member to a set; SCARD, which tells us how many members are in a set; SISMEMBER, which checks if a member is in a set; and SINTER, which gives us the intersection of one or more sets. If you want to learn more about sets and other Redis data types, check out our free course: RU101 - Introduction to Redis Data Structures, at Redis University. Thanks for watching and see you next time. :

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)